# ML Loto Pred

#### Librerias

In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

## 1- Read data

In [2]:
data = data= pd.read_excel('base.xlsx', sheet_name='datos')
data.tail()

,Fecha,bolo1,bolo2,bolo3,bolo4,bolo5,bolo6,Loto +.1,Loto +.2
317,2022-07-23,9,13,16,22,24,38,6,15
318,2022-07-27,4,5,12,20,21,37,8,1
319,2022-07-30,9,13,14,15,29,36,5,7
320,2022-08-03,14,20,27,33,34,36,6,1
321,2022-08-06,1,4,6,7,24,29,1,4


## 2- data Prep

In [3]:
# set time to index
data.set_index('Fecha',inplace = True )

In [4]:
# make fetures dates
miercoles = 3
sabados = 4

In [5]:
bol1 = data.copy()
bol1['prediction'] = bol1[['bolo1']].shift(-1)
bol1.head()

,bolo1,bolo2,bolo3,bolo4,bolo5,bolo6,Loto +.1,Loto +.2,prediction
Fecha,,,,,,,,,
2019-04-10,12,15,18,23,25,31,4,1,7.0
2019-04-13,7,9,10,21,27,28,5,5,1.0
2019-04-17,1,3,13,20,35,36,1,13,4.0
2019-04-20,4,6,11,13,27,28,10,7,1.0
2019-04-24,1,7,19,33,36,38,1,2,9.0


In [6]:
bol1.tail()

,bolo1,bolo2,bolo3,bolo4,bolo5,bolo6,Loto +.1,Loto +.2,prediction
Fecha,,,,,,,,,
2022-07-23,9,13,16,22,24,38,6,15,4.0
2022-07-27,4,5,12,20,21,37,8,1,9.0
2022-07-30,9,13,14,15,29,36,5,7,14.0
2022-08-03,14,20,27,33,34,36,6,1,1.0
2022-08-06,1,4,6,7,24,29,1,4,NaN


## 3- Fetures selection
- X and y
- Split into train and test

In [7]:
X = bol1.drop('prediction', axis=1 )
y = bol1['prediction']
X.shape, y.shape

((322, 8), (322,))

In [8]:
X = X[:-1]
y = y[:-1]

In [9]:
#split data
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = .30, random_state=42)

## 3- Modelos

### 3.1- Linear

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
lr = LogisticRegression(max_iter=300, solver='liblinear')
lr.fit(X_train, y_train)

LogisticRegression(max_iter=300, solver='liblinear')

In [13]:
y_pred = lr.predict(X_test)

In [14]:
# Evaluation 
metrics.r2_score(y_test, y_pred)

-2.873591872984216

### 3.2-Tree

In [15]:
from sklearn.tree import DecisionTreeRegressor

In [16]:
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

DecisionTreeRegressor()

In [17]:
#predictions
y_dt = dt.predict(X_test)
# Evaluation 
metrics.r2_score(y_test, y_dt)

-2.200285131113569

### 3.3 Random Forest

In [18]:
from sklearn.ensemble import RandomForestRegressor

In [19]:
rf = RandomForestRegressor(n_estimators=500)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=500)

In [20]:
y_rf = rf.predict(X_test)
# Evaluation 
metrics.r2_score(y_test, y_rf)

-0.3352240233714028

### 3.5 XGBOOST

In [21]:
from xgboost import XGBRFRegressor

In [22]:
xgb = XGBRFRegressor()
xgb.fit(X_train, y_train)

XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bytree=1, enable_categorical=False, gamma=0, gpu_id=-1,
               importance_type=None, interaction_constraints='',
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=4,
               num_parallel_tree=100, objective='reg:squarederror',
               predictor='auto', random_state=0, reg_alpha=0,
               scale_pos_weight=1, tree_method='exact', validate_parameters=1,
               verbosity=None)

In [23]:
y_xgb = xgb.predict(X_test)

In [24]:
metrics.r2_score(y_test, y_xgb)

-0.31727451165410425

## 4- Sample Predictions

In [25]:
result = dt.predict([[9,10,12,16,30,37,7,3]])
round(result[0])

9

In [26]:
result = lr.predict([[9,10,12,16,30,37,7,3]])
round(result[0])

3

In [27]:
result = rf.predict([[9,10,12,16,30,37,7,3]])
round(result[0])

8

la precicción del modelo Linear es mas exacta